In [1]:
import pandas as pd 
from db import *
# from config_poc import *

pd.set_option('display.max_columns', None)

In [2]:
print(TABLE_RELATION)

g_relation


In [6]:
primary_key = 'http://people.ece.cornell.edu/atang/'
ref_tab=TABLE_PERSON
ref_key="url"
ref_val=primary_key
inter_table_name=TABLE_RELATION
table_name=TABLE_WORK
rel_type='person-work'

In [7]:
with DBConn() as _conn:
    # fetch child table keys first
    sql_stmt = f"""
        select 
            it.ref_key_sub as "key_col", 
            it.ref_val_sub as "key_val"
        from {inter_table_name} it 
        where it.rel_type = '{rel_type}'
            and it.ref_tab = '{ref_tab}'
            and it.ref_key = '{ref_key}'
            and it.ref_val = '{ref_val}'
            and it.ref_tab_sub = '{table_name}'
    """
    df = pd.read_sql(sql_stmt, _conn)  

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_11704\695382134.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [8]:
df

,key_col,key_val
0,id,atang/pub/22/ICML2022.pdf
1,id,atang/pub/21/NeurIPS-2021.pdf
2,id,b62cf974-c5e4-494a-a29c-e34576bb1ec2


In [9]:
row_list = df.groupby("key_col")["key_val"].apply(list)

In [12]:
row_list.to_dict()

{'id': ['atang/pub/22/ICML2022.pdf',
  'atang/pub/21/NeurIPS-2021.pdf',
  'b62cf974-c5e4-494a-a29c-e34576bb1ec2']}

In [15]:
where_clause = []
for k,v in row_list.items():
    where_clause.append(f" {k} in {str(v).replace('[','(').replace(']',')')}")

In [16]:
where_clause

[" id in ('atang/pub/22/ICML2022.pdf', 'atang/pub/21/NeurIPS-2021.pdf', 'b62cf974-c5e4-494a-a29c-e34576bb1ec2')"]

In [17]:
with DBConn() as _conn:
    sql_stmt = f"""
        select *
        from {table_name}
        where {" or ".join(where_clause)}
    """
    df = pd.read_sql(sql_stmt, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_11704\63403375.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [18]:
df

,id,uid,ts,work_type,name,url,note,summary,authors,tags
0,atang/pub/22/ICML2022.pdf,0,2023-04-25 22:39:21.303-04,publication,Task-aware Privacy Preservation for Multi-dime...,http://people.ece.cornell.edu/atang/pub/22/ICM...,None,Extensive experiments demonstrate that our tas...,"J. Cheng, A. Tang and S. Chinchali",None
1,atang/pub/21/NeurIPS-2021.pdf,0,2023-04-25 22:39:21.303-04,publication,Data Sharing and Compression for Cooperative N...,http://people.ece.cornell.edu/atang/pub/21/Neu...,None,we present theoretical compression results for...,"J. Cheng, M. Pavone, S. Katti, S. Chinchali, a...",None
2,b62cf974-c5e4-494a-a29c-e34576bb1ec2,0,2023-04-25 22:39:21.303-04,publication,On upper bounding Shannon capacity of graph th...,http://people.ece.cornell.edu/atang/pub/19/opt...,None,"In this paper, we consider the\npossibility o...","Yingjie Bi, Ao Tan",None


In [3]:
from datetime import datetime, date 

In [6]:
datetime.now().date(), datetime.now().time()

(datetime.date(2023, 4, 30), datetime.time(13, 18, 10, 181228))